In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import math

pd.set_option('display.max_columns', None)

In [2]:
os.chdir('/Users/이진우/Desktop/학회/4기활동/심화스터디, 세션/Network_Analysis/플젝')

# 서울버스노선 정보 (노선명, 정류장id, 같은 노선 내에 정류장 순서, 좌표)
bus = pd.read_csv('./서울노선정보.csv')
bus = bus.loc[:,['노선명', 'ARS-ID', '순번','X좌표','Y좌표']]

# 2020년 1월 승하차 정보
b1 = pd.read_csv('./1.csv', encoding='cp949')

### bus : 특정 노선 내 정류장들의 순서와 좌표에 대한 데이터

In [3]:
bus.head(3)

,노선명,ARS-ID,순번,X좌표,Y좌표
0,0017,3689,1,126.946517,37.534363
1,0017,3298,2,126.949304,37.533961
2,0017,3321,3,126.950449,37.533744


### b1 : 특정 노선 내 정류장에 대한 승하차수 데이터가 있다
- 이때 버스정류장ARS번호 = ARS-ID

In [4]:
b1.head(3)

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20200101,N62,N62번(양천공영차고지~면목동차고지),04133,왕십리역,0,13,20200104
1,20200101,N62,N62번(양천공영차고지~면목동차고지),04136,행당1동주민센터.성동소방서,2,3,20200104
2,20200101,N62,N62번(양천공영차고지~면목동차고지),04186,성동교남단,0,2,20200104


# Edge 파일과 Node 파일을 만들기 위한 전처리
- 전처리하여 만들어줄 데이터프레임의 칼럼 : 
- '노선',  '정류장id',  'label',  '같은 노선 내 정류장 순서',  '지도좌표',  '승하차'

### 특정 노선 안의 버스정류장에 대한 승하차 데이터가 30일 동안 있을거니까
- 승하차수는 평균내주자
- 물론 역명은 딱 하나만 필요하니 'first'로 처리

In [5]:
B = b1.groupby(['노선번호','버스정류장ARS번호']).agg({'역명':'first','승차총승객수':'mean','하차총승객수':'mean'}).reset_index()
B.head(3)

,노선번호,버스정류장ARS번호,역명,승차총승객수,하차총승객수
0,0017,03001,한강대교북단.LG유플러스,219.193548,416.193548
1,0017,03003,신용산역,146.548387,381.838710
2,0017,03122,한강대교북단,198.032258,147.354839


In [6]:
# 정류장id중에 '~' 인것도 있는데 정체는 가상id
# 가상id란? (특정 노선의 정류장 간격이 엄청 길때 기록을 위해 일부러 만든 가상id)
# 즉, 필요없으니까 없애주자
B = B.drop(B[B['버스정류장ARS번호']=='~'].index)

In [7]:
# 정류장id를 int로 통일하자아
B['버스정류장ARS번호'] = B['버스정류장ARS번호'].astype(int)

In [8]:
# 특정 노선에 대해 정류장id에 대해 역명이 혹여나 겹치는게 있는지 확인
B.duplicated(['노선번호','버스정류장ARS번호','역명']).sum()
# 다행이 없다

0

In [9]:
# 이제 bus와 B를 merge시키자
# merge하기위한 key : 노선, 정류장id
T = pd.merge(bus, B, how='left', left_on=['노선명','ARS-ID'], right_on=['노선번호', '버스정류장ARS번호'])
T.head(3)

,노선명,ARS-ID,순번,X좌표,Y좌표,노선번호,버스정류장ARS번호,역명,승차총승객수,하차총승객수
0,0017,3689,1,126.946517,37.534363,0017,3689.0,청암자이아파트,19.516129,0.225806
1,0017,3298,2,126.949304,37.533961,0017,3298.0,청암동강변삼성아파트,59.951613,58.096774
2,0017,3321,3,126.950449,37.533744,0017,3321.0,청심경로당,94.322581,1.967742


In [10]:
# merge하면서 생긴 NaN값 객체들 없애주자아
T = T.dropna(axis=0)

In [11]:
# 이름만 다르고 같은 데이터 (노선명, 노선번호 등) 를 없애주자
T = T.loc[:,['노선명','ARS-ID','역명','순번','X좌표','Y좌표','하차총승객수','승차총승객수']]

# 같은 노선에 대하여 정류장 순서대로 sort해주자
# 그래야 edge 파일 만들때 편하다
T.sort_values(['노선명','순번'], inplace=True)
T.reset_index(drop=True, inplace=True)
T.head(3)

# 노선명과 순번 값을 주목하자아ㅏ!

,노선명,ARS-ID,역명,순번,X좌표,Y좌표,하차총승객수,승차총승객수
0,0017,3689,청암자이아파트,1,126.946517,37.534363,0.225806,19.516129
1,0017,3298,청암동강변삼성아파트,2,126.949304,37.533961,58.096774,59.951613
2,0017,3321,청심경로당,3,126.950449,37.533744,1.967742,94.322581


# Edge 파일 만들기-1 (weight = 승차수)
- 최종파일 형태 : source, target, weight

In [12]:
# 서울 내 unique한 노선명들을 list로 담아두자,
LANE = T.노선명.unique().tolist()
# 최종파일 : result
result = pd.DataFrame(columns=['source','target','weight'])

for lane in LANE:
    # 특정 노선에 대한 데이터만 담자 : lane_df
    lane_df = T.loc[T['노선명']==lane].reset_index(drop=True)
    # 그 노선 내 버스가 지나가는 총 정류장수 (회차하면 중복정류장도 있겠지)
    length_of_lane = len(lane_df)
    
    i=0 # 순번끼리 잇기 위해 준비한 친구
    
    while i < (length_of_lane - 1) : # 그 노선 내 마지막 순번될때까지 loop
        # 만들어줄 데이터 형태 : [source, target, weight, lane]
        # source : ith 정류장
        # target : (i+1)th 정류장
        source = lane_df.at[i,'ARS-ID']
        target = lane_df.at[i+1, 'ARS-ID']
        weight = round(lane_df.at[i, '승차총승객수'])
        
        # [source, target, weight, lane] 하나하나 만들어 result에 담아주자
        tmp = {'source':source, 'target':target, 'weight':weight}
        result = result.append(tmp, ignore_index=True)
        # 그 다음 순번으로 갱신
        i = i+1

# 빠짐없이 만들어졌는지 확인
# 이때 T.노선명.nunique() 빼주는 이유는
# source, target 만들어주면 그만큼 원래 T에서 1개씩 줄어드니까
# ex) 순번 0,1,2 : 3개 // (0,1),(1,2) : 2개
print(T.shape[0]-T.노선명.nunique(), result.shape[0])
result.head(3)

35365 35365


,source,target,weight
0,3689.0,3298.0,20.0
1,3298.0,3321.0,60.0
2,3321.0,3304.0,94.0


In [13]:
# 완성
result.to_csv('Edge_sample.csv', index=False)

# Edge 파일 만들기-2 (weight = 570m / distance)
- 최종파일 형태 : source, target, distance, weight
- distance : haversine 패키지 이용하여 지도좌표들간 거리 구해줄것이당
- 그 즉슨 패키지 설치해주셔야함...
- 참고로 코드실행이 쬐금 오래걸림

In [14]:
from haversine import haversine

LANE = T.노선명.unique().tolist()
result = pd.DataFrame(columns=['source','target','distance','weight'])

for lane in LANE:
    lane_df = T.loc[T['노선명']==lane].reset_index(drop=True)
    length_of_lane = len(lane_df)
    
    i=0
    
    while i < (length_of_lane - 1):
        # 만들어줄 데이터 형태 : [source, target, distance, weight]
        source = lane_df.at[i,'ARS-ID']
        target = lane_df.at[i+1, 'ARS-ID']
        
        # 두개의 좌표간 거리 구할려면
        # (latitude(Y좌표), longitude(X좌표)) 순서로 계산해야함
        # 이떄 haversine의 결과값의 단위는 km인거 명심하자!
        source_coordinate = lane_df.loc[i,['Y좌표','X좌표']]
        target_coordinate = lane_df.loc[i+1,['Y좌표','X좌표']]
        distance = haversine(source_coordinate,target_coordinate)
        
        # weight는 거리가 짧을수록 커야하니까 distance의 역수를 취해주되
        # 570m 이하의 거리의 weight : 1
        # 570m 이상의 거리의 weight : 0.57km / distance 해주자
        if distance > 0.57:
            weight = 0.57 / distance
        elif distance <= 0.57:
            weight = 1
        
        # [source, target, distance, weight] 하나하나 만들어 result에 담아주자
        tmp = {'source':source, 'target':target, 'distance':distance, 'weight':weight}
        result = result.append(tmp, ignore_index=True)
        # 그 다음 순번으로 갱신
        i = i+1

# 빠짐없이 만들어졌는지 확인
print(T.shape[0]-T.노선명.nunique(), result.shape[0])
result.head(3)

35365 35365


,source,target,distance,weight
0,3689.0,3298.0,0.249719,1.0
1,3298.0,3321.0,0.103780,1.0
2,3321.0,3304.0,0.071748,1.0


In [15]:
# 완성
result.to_csv('Edge_sample2.csv', index=False)

# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

# Node 파일 만들기
- 칼럼 : id, label, 지도좌표
- 전략 : 한번에 Node파일 만들기 힘들어서
- id,label만 있는 데이터프레임 따로 만들고
- id, 지도좌표 만 있는 데이터프레임도 따로 만들어서
- 두 데이터프레임을 id기준으로 merge해버리자

### 첫번째 데이터프레임 : id, label

In [16]:
Simple_node = T.groupby('ARS-ID')['역명'].unique().reset_index()
Simple_node.head(3)

,ARS-ID,역명
0,1001,[종로2가사거리]
1,1002,[창경궁.서울대학교병원]
2,1003,[명륜3가.성대입구]


In [17]:
# groupby()[].unique() 함수의 문제점은
# 결과가 위에처럼 리스트 형태로 저장된다
# 리스트 형태를 없애기 위해 어쩔수 없이 밑에 코드로 처리해준다.

tmp = []
for idx in Simple_node.index:
    # 리스트의 값을 꺼내어 tmp에 담는것
    tmp.append(Simple_node.역명[idx][0])

# tmp를 고대로 새로운 칼럼 label에 붙여버린다.
Simple_node['label']=tmp

### 두번째 데이터프레임 : id, 지도좌표

In [18]:
# 이제 지도좌표

bus_geo = T.loc[:,['ARS-ID','X좌표','Y좌표']]

# 애초에 T에 모든 버스노선이 있는만큼
# 버스정류장 중복이 겁나 많은데
# 현재 Node 파일 만드는게 목표이니까 중복 없애주자
# keep='first'는 중복 데이터들 중에 맨 첫번째것만 살리자는것
bus_geo = bus_geo.drop_duplicates(['ARS-ID'], keep='first')

In [19]:
Simple_node = pd.merge(Simple_node, bus_geo, how='left', on='ARS-ID')
Simple_node.head(3)

,ARS-ID,역명,label,X좌표,Y좌표
0,1001,[종로2가사거리],종로2가사거리,126.987786,37.569764
1,1002,[창경궁.서울대학교병원],창경궁.서울대학교병원,126.996520,37.579179
2,1003,[명륜3가.성대입구],명륜3가.성대입구,126.998290,37.582709


In [20]:
# 역명은 필요없으니 없애주자
Simple_node = Simple_node.loc[:,['ARS-ID', 'label', 'X좌표', 'Y좌표']]
# 인덱스를 label로 만들어줄 계획
Simple_node.set_index('label', inplace=True)
# 칼럼 이름 바꾸기
Simple_node.rename(columns={'ARS-ID':'id', 'X좌표':'longitude', 'Y좌표':'latitude'}, inplace=True)

# 최종 형태 :
Simple_node.head(3)

,id,longitude,latitude
label,,,
종로2가사거리,1001,126.987786,37.569764
창경궁.서울대학교병원,1002,126.996520,37.579179
명륜3가.성대입구,1003,126.998290,37.582709


In [21]:
# 완성
Simple_node.to_csv('Node_file.csv', sep=',')